# Instructions for Freesurfer

In [ ]:
import os 
import shutil
import glob

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
stats1 = pd.read_csv('../../secret_data/aseg_stats.txt', sep ='\t')
stats1

In [ ]:
stats2 = pd.read_csv('../../secret_data/aseg_stats2.txt', sep ='\t')
stats2

In [ ]:
total_stats = pd.concat([stats2, stats1])

In [ ]:
#total_stats.to_csv('../../secret_data/brain_volumes_from_freesurfer_no_qc.csv')

In [ ]:
total_stats.columns

In [ ]:
total_stats = total_stats.rename(columns= {'Measure:volume':'Participant ID'})
total_stats['Participant ID'] = total_stats['Participant ID'].str.replace('/', '')

In [ ]:
total_stats.head(4)

In [ ]:
total_stats.to_csv('../../secret_data/brain_volumes_from_freesurfer_no_qc.csv')

In [ ]:
#total

In [ ]:
excel = pd.read_excel("../../secret_data/BRICK_datums_scans_clean_17072024.xlsx")
excel.head(3)


In [ ]:
excel.columns


In [ ]:
total_stats_numeric = total_stats.drop(['Participant ID'], axis=1)
total_stats_numeric.corr() 
#.corr()

In [ ]:
sns.heatmap(total_stats_numeric.corr(method='spearman', numeric_only=False))

Let's ask if left and right correlate...or other things.

In [ ]:
left_to_right = total_stats_numeric[['Left-Lateral-Ventricle', 'Left-Inf-Lat-Vent',
       'Left-Cerebellum-White-Matter', 'Left-Cerebellum-Cortex',
       'Left-Thalamus-Proper', 'Left-Caudate', 'Left-Putamen', 'Left-Pallidum',
        'Left-Hippocampus',
       'Left-Amygdala', 'Left-Accumbens-area', 'Left-VentralDC',
       'Left-vessel', 'Left-choroid-plexus', 'Right-Lateral-Ventricle',
       'Right-Inf-Lat-Vent', 'Right-Cerebellum-White-Matter',
       'Right-Cerebellum-Cortex', 'Right-Thalamus-Proper', 'Right-Caudate',
       'Right-Putamen', 'Right-Pallidum', 'Right-Hippocampus',
       'Right-Amygdala', 'Right-Accumbens-area', 'Right-VentralDC',
       'Right-vessel', 'Right-choroid-plexus', 
        'Left-WM-hypointensities',
       'Right-WM-hypointensities', 
       'Left-non-WM-hypointensities', 'Right-non-WM-hypointensities',]]
sns.heatmap(left_to_right.corr(method='spearman', numeric_only=False))

Yes, left and right correlate. Now let's ask whether wmh increase with age....

In [ ]:
total_stats_numeric.columns

In [ ]:
total_stats

In [ ]:
wmh_info = total_stats[[ 'Participant ID','3rd-Ventricle', '4th-Ventricle', 'Brain-Stem', 'Left-Hippocampus',
       'Left-Amygdala', 'CSF',  'Right-Lateral-Ventricle',
       'Right-Inf-Lat-Vent', 'Right-Cerebellum-White-Matter',
       'Right-Cerebellum-Cortex', 'Right-Thalamus-Proper', 'Right-Caudate',
       'Right-Putamen',  'Right-Hippocampus',
       'Right-Amygdala',  'Right-VentralDC',
       'Right-choroid-plexus',
       'WM-hypointensities', 'Left-WM-hypointensities',
       'Right-WM-hypointensities', 'non-WM-hypointensities',
       'Left-non-WM-hypointensities', 'Right-non-WM-hypointensities',]]

In [ ]:
wmh_info

1. first use python to move files a few at a time.

In [ ]:
find /mnt/data/output -iwholename '*Freesurfer*/*.nii.gz' -type f

In [ ]:
shutil.copy?

In [ ]:
glob.glob?

In [ ]:
_files = glob.glob('C:/presentations/**', recursive=True)

In [ ]:
#_files

In [ ]:
for fileName_relative in glob.glob('C:/experimental', *, recursive=True):       ## first get full file name with directores using for loop

    print("Full file name with directories: ", fileName_relative)

In [ ]:
def find_surf_and_move(src, dst, filestring):
    if not os.path.exists(dst):
        os.mkdir(dst)
    files = glob.glob(os.path.join(src,'**'), recursive=True)

    # for path, subdirs, files in os.walk(src):
    #     print(files)
    #     for name in files:
    #         if filestring in name:
    #         #if fnmatch(name, pattern):
    #             full_name = os.path.join(path, name)
    #             #destination_folder = 
    #             print(name)

                #shutil.copytree(real_name, dst)
    
    
    
    
#     files = os.listdir()
    
#     if filestring is in file:
        
#     shutil.copytree(src, dst) 

In [ ]:
find_surf_and_move(file_path, 'C:/experimental', 'cvasl')

In [ ]:
root = file_path
#pattern = "*."

for path, subdirs, files in os.walk(root):
    for name in files:
        if 'cvasl' in name:
            #if fnmatch(name, pattern):
            print(os.path.join(path, name))